In [1]:
%matplotlib inline
from spectral_cube import SpectralCube
from aplpy import FITSFigure
import numpy as np
import os
from astropy.io import fits
import warnings
import numpy as np
import shells
import shell_model
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

SyntaxError: invalid syntax (shells.py, line 630)

## Fixing Model Cloud Parameters for Orion A
distance: 414 pc (Menten+ 2007)

thickness: 1 pc (upper limit: ~ largest shell radius

fwhm: 4 km/s (from maps below)

beta: 2 power spectrum (http://adsabs.harvard.edu/abs/1994ApJ...429..645M)

### Calculate Velocity FWHM from NRO Data

In [ ]:
cubefile12co = "../nro_maps/12CO_20161017_FOREST-BEARS_spheroidal_xyb_grid7.5_0.099kms_YS.fits"
regionfile = "../shell_candidates/AllShells.reg"

In [ ]:
cube = SpectralCube.read(cubefile12co)
mom0_map = cube.moment0()
fwhm_map = cube.linewidth_fwhm() 

In [ ]:
print(np.nanmedian(fwhm_map))

fig = FITSFigure(fwhm_map.hdu)
fig.show_grayscale(vmin=0, vmax=10000)
fig.recenter(83.72707, -5.07792, 0.1)
fig.show_colorbar()
fig.colorbar.set_axis_label_text("FWHM (m/s)")

### Smooth the Velocity FWHM Map For a Global Estimate of FWHM in Orion A

In [ ]:
from scipy.ndimage.filters import gaussian_filter
fwhm_smooth = gaussian_filter(fwhm_map.data, 1)
print(np.nanmedian(fwhm_smooth))
fig = FITSFigure(fwhm_smooth)
fig.show_grayscale()
#fig.recenter(83.72707, -5.07792, 0.1)
fig.show_colorbar()
fig.colorbar.set_axis_label_text("FWHM (m/s)")

### Plot an example turbulent model cube.

In [ ]:
# For Shell 18
outfile = '../turbulent_model/shell18_ppv.fits'
model_pars = {
    'outfile':"'{}'".format(outfile),
    'dist':414, # pc
    'pix_size':7.5, # arcsec
    'vstep':0.099, # km/s
    'acen':83.72707, # deg
    'dcen':-5.07792, # deg
    'thickness':1., # pc
    'fwhm':4.0, # km/s
    'beta':0.0, # spectral index
    'r':0.31, # pc
    'dr':0.2, # pc
    'vexp':1.6, # km/s
    'depth_offset':0.0, # pc
    'vel_offset':0.0, # km/s
    'v0':13.6 # km/s
             }
idl_arg_list = ["{}={}".format(k, v) for k, v in model_pars.items()]

idl_arg_string = ', '.join(idl_arg_list)
#call_string = "idl << EOF \n" +\
#              ".r turbulent_model \n" +\
#              "turbulent_model, {} \n".format(idl_arg_string) +\
#              "EOF"
call_string = 'idl -e "turbulent_model, {}"'.format(idl_arg_string)

In [ ]:
#Call the idl program turbulent_model.pro with the above parameters
e = os.system(call_string)

### Integrated Intensity (Column Density) of Model Shell

In [ ]:
model_cube = SpectralCube.read(outfile)
model_mom0 = model_cube.moment0()

In [ ]:
fig = FITSFigure(model_mom0.hdu)
fig.show_grayscale()
cb = fig.show_colorbar()
fig.set_title("r = {}, dr = {}, vexp = {}".format(model_pars['r'], model_pars['dr'], model_pars['vexp']))

### Extract a subcube around an Orion A shell

In [ ]:
n = 17
shell_list = shells.get_shells()
subcubes = shells.subcubes_from_ds9(cube=cubefile12co, region_file=regionfile, pad_factor=1.5, shape='square')
shell = shell_list[n]
shell_subcube = subcubes[n].spectral_slab(shell.vmin, shell.vmax)
shell_mom0 = shell_subcube.moment0()

In [ ]:
fig = FITSFigure(shell_mom0.hdu)
fig.show_grayscale()
cb = fig.show_colorbar()
fig.set_title("Shell 18 12CO: r = {}, dr = {}, vexp = {}".format(model_pars['r'], model_pars['dr'], model_pars['vexp']))
fig.show_contour(model_mom0.hdu, levels=10)

In [ ]:
model_profile, model_profile_error, model_bins = shell_model.radial_profile(np.array(model_mom0), nbins=30)
shell_profile, shell_profile_error, shell_bins = shell_model.radial_profile(np.array(shell_mom0), nbins=30)

In [ ]:
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(121)
ax1.errorbar(model_bins*model_pars['pix_size']/60., model_profile,
             yerr=model_profile_error, linestyle="dotted", label="S.E.M.")
ax1.set_xlabel("Radius [arcmin]")
ax1.set_ylabel("Averaged Integrated Intensity")
ax1.set_title("Radial Profile of Model: r = {}, dr = {}".format(model_pars['r'], model_pars['dr']))

ax2 = fig.add_subplot(122)
ax2.errorbar(shell_bins*model_pars['pix_size']/60., shell_profile,
             yerr=shell_profile_error, linestyle="dotted", label="S.E.M.")
ax2.set_xlabel("Radius [arcmin]")
ax2.set_ylabel("Averaged Integrated Intensity [K m/s]")
ax2.set_title("Radial Profile of Shell 18".format(model_pars['r'], model_pars['dr']))

### Plot the normalized profiles against each other.

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.plot(model_bins*model_pars['pix_size']/60., model_profile/np.nanmax(model_profile),
             linestyle="dotted", label="Model")
plt.plot(shell_bins*model_pars['pix_size']/60., shell_profile/np.nanmax(shell_profile),
             linestyle="dashed", label="Shell 18")
plt.xlabel("Radius [arcmin]")
plt.ylabel("Averaged Integrated Intensity")
plt.title("Radial Profile of Model: r = {}, dr = {}".format(model_pars['r'], model_pars['dr']))